[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/shhommychon/DeZero-Koki/blob/master/from_scratch_3/steps/step11.ipynb)

# 제1고지 미분 자동 계산

In [1]:
import numpy as np


class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f"{type(data)}은(는) 지원하지 않습니다.")

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None:
                funcs.append(x.creator)


In [2]:
import numpy as np


def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x


In [3]:
# class Square(Function):
#     def forward(self, x):
#         return x ** 2

#     def backward(self, gy):
#         x = self.input.data
#         gx = 2 * x * gy
#         return gx


# class Exp(Function):
#     def forward(self, x):
#         return np.exp(x)

#     def backward(self, gy):
#         x = self.input.data
#         gx = np.exp(x) * gy
#         return gx


# def square(x):
#     return Square()(x)  # 한 줄로 작성


# def exp(x):
#     return Exp()(x)


In [4]:
def numerical_diff(f, x, eps=1e-4):
    """

      param:
        f (Function): 미분의 대상이 되는 함수
        x (Variable): 미분을 계산하는 변수
        eps (float): 작은 값
    """
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2 * eps)

# 제2고지 자연스러운 코드로

## 11단계 가변 길이 인수(순전파 편)

### 11.1 Function 클래스 수정

In [5]:
class Function:
    """

    * Function 클래스는 기반 클래스로서, 모든 함수에 공통되는 기능을 구현합니다.
    * 구체적인 함수는 Function 클래스를 상속한 클래스에서 구현합니다.
    """
    def __call__(self, input):
        x = input.data  # (1) Variable에서 실제 데이터를 꺼냄.
        y = self.forward(x)  # (2) 구체적인 계산 진행.
        output = Variable(as_array(y))  # (3) 계산 결과를 Variable에 넣음.
        output.set_creator(self)  # (4) 본 함수가 계산 결과의 '창조자'임을 명시.
        self.input = input
        self.output = output
        return output

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()

In [6]:
class Function:
    """

    * Function 클래스는 기반 클래스로서, 모든 함수에 공통되는 기능을 구현합니다.
    * 구체적인 함수는 Function 클래스를 상속한 클래스에서 구현합니다.
    """
    def __call__(self, inputs):
        xs = [ x.data for x in inputs ]  # (1)
        ys = self.forward(xs)  # (2)
        outputs = [ Variable(as_array(y)) for y in ys ]  # (3)
        for output in outputs: output.set_creator(self)  # (4)
        self.inputs = inputs
        self.outputs = outputs
        return outputs

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

### 11.2 Add 클래스 구현

In [7]:
class Add(Function):
    def forward(self, xs):
        x0, x1 = xs
        y = x0 + x1
        return y,

In [8]:
xs = [Variable(np.array(2)), Variable(np.array(3))]
f = Add()
ys = f(xs)  # ys 튜플
y = ys[0]
print(y.data)

5
